## Importamos las librerias necesarias para empezar las transformaciones de los datos

In [1]:
import pandas as pd
import numpy as np
import ast
import yellowbrick as yw
import json as js
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
warnings.simplefilter("ignore")

In [2]:
filas = []

with open('output_steam_games.json','r',encoding='utf-8') as file:
    for line in file.readlines():
        filas.append(js.loads(line)) # Como los tipo de valores no eran strings que es el tipo esperado por ast.litreal_eval, usamos json.loads

df_games = pd.DataFrame(filas)

In [3]:
df_games.head()
# Observamos los datos

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_games.shape

(120445, 13)

In [5]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


### Veremos cuantas filas serán eliminadas por tener data irrelevante

In [6]:
df_games[df_games.isnull().sum(axis=1)==13]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Observamos que de toda la data, existen 88310 de 120445 filas,con data totalmente irrelevante

df_games = df_games[df_games.isnull().sum(axis=1)!=13]
df_games.reset_index(drop=True, inplace=True) # Reseteamos los indices de fila por haber eliminado 1 fila, lo cual altera el orden
df_games
# Este nuevo dataset ahora contiene 32135 filas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [8]:
df_games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

### Volvemos a verificar si existe alguna fila que sea irrelevante y que contenga data relevante

In [9]:
df_mas = df_games[df_games.isnull().sum(axis=1)>6]
df_mas

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN


In [10]:
df_mas['url']

74    http://store.steampowered.com/
Name: url, dtype: object

In [11]:
df_games = df_games[df_games.isnull().sum(axis=1)<=6]
df_games.reset_index(drop=True, inplace=True)  

# Reseteamos los indices de fila por haber eliminado 1 fila, lo cual altera el orden

## Hemos eliminado todas las filas co elementos que no aportan, tambien las 13 columnas NAN, ahora observaremos las filas con 6 valores de NaN por fila

In [12]:
df_filtrante = df_games[df_games.isnull().sum(axis=1)>=6]
df_filtrante

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
36,NaN,NaN,Kingdom Watcher,NaN,http://store.steampowered.com/app/680970/Kingd...,NaN,"[Action, Free to Play, Strategy]",http://steamcommunity.com/app/680970/reviews/?...,"[Single-player, Online Multi-Player, HTC Vive,...",NaN,False,680970,NaN
45,NaN,NaN,Panoptic,NaN,http://store.steampowered.com/app/541930/Panop...,NaN,"[Early Access, Indie, Action, VR]",http://steamcommunity.com/app/541930/reviews/?...,"[Multi-player, Local Multi-Player, Partial Con...",NaN,True,541930,NaN
53,NaN,NaN,Golf Masters,NaN,http://store.steampowered.com/app/374970/Golf_...,NaN,"[Simulation, Sports, Casual, Indie, Golf, VR]",http://steamcommunity.com/app/374970/reviews/?...,"[Single-player, Steam Achievements, Partial Co...",NaN,False,374970,NaN
2541,NaN,NaN,NoLimits 2 Roller Coaster Simulation Demo,NaN,http://store.steampowered.com/app/319260/NoLim...,NaN,"[Simulation, Indie]",http://steamcommunity.com/app/319260/reviews/?...,"[Single-player, Game demo, HTC Vive, Oculus Ri...",NaN,False,319260,NaN
5372,NaN,NaN,BrainBread 2 Mod Tools,BrainBread 2 Mod Tools,http://store.steampowered.com/app/382990/Brain...,2016-02-17,NaN,http://steamcommunity.com/app/382990/reviews/?...,NaN,NaN,False,382990,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31935,NaN,NaN,Fruit Golf,NaN,http://store.steampowered.com/app/406890/Fruit...,NaN,"[Casual, Sports, Indie, VR]",http://steamcommunity.com/app/406890/reviews/?...,"[Single-player, Multi-player, Shared/Split Scr...",NaN,False,406890,NaN
31991,NaN,NaN,Mutato Match,NaN,http://store.steampowered.com/app/391480/Mutat...,NaN,"[Early Access, Indie, Casual, VR]",http://steamcommunity.com/app/391480/reviews/?...,"[Single-player, HTC Vive, Oculus Rift, Tracked...",NaN,True,391480,NaN
32053,NaN,NaN,VR Dunhuang,NaN,http://store.steampowered.com/app/752200/VR_Du...,NaN,"[Adventure, RPG, Casual, Simulation]",http://steamcommunity.com/app/752200/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",NaN,False,752200,NaN
32056,NaN,NaN,COMPLEX a VR Puzzle Game,NaN,http://store.steampowered.com/app/772420/COMPL...,NaN,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/772420/reviews/?...,"[Single-player, HTC Vive, Oculus Rift, Tracked...",NaN,False,772420,NaN


## Vimos coincidencia de nombres entre las columnas app_name y title en el df original, por lo tanto una de las columnas desaparecera

In [13]:
df_filtrante.isnull().sum()

# Aca observamos que la que mas tiene valores nulos es title, por lo tanto esa fila sera la que eliminaremos

publisher       121
genres          121
app_name          0
title           117
url               0
release_date    117
tags              5
reviews_url       0
specs             4
price           120
early_access      0
id                0
developer       121
dtype: int64

In [14]:
df_games.drop(['title'], inplace=True, axis=1)
df_games

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
32129,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
32130,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
32131,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32132,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [15]:
df_games.isnull().sum()

publisher       8051
genres          3282
app_name           1
url                0
release_date    2066
tags             162
reviews_url        1
specs            669
price           1377
early_access       0
id                 1
developer       3298
dtype: int64

### En la consigna se nos pide un filtro de desarrollar, a la vez observamos que  la columna de url es irrelevante, pero antes observamos que una fila no tiene appname, la imputaremos con su respectivo nombre relacionado a la URL a la que corresponde

### Imputamos los valores faltantes en la columna id y app_name

In [16]:
df_id_faltante = df_games[df_games['id'].isnull()]
df_id_faltante

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
30960,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [17]:
df_app_faltante = df_games[df_games['app_name'].isnull()]
df_app_faltante

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
2579,NaN,"[Action, Indie]",NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,NaN


In [18]:
df_app_faltante['reviews_url']

2579    http://steamcommunity.com/app/317160/reviews/?...
Name: reviews_url, dtype: object

In [19]:
type(df_id_faltante.loc[30960,'app_name'])

str

In [20]:
df_games.at[30960,'id'] = 200260
df_games.at[2579,'app_name'] = 'Duet'
df_games.drop(['reviews_url','early_access'],inplace = True ,axis=1)
df_games

,publisher,genres,app_name,url,release_date,tags,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,767400,彼岸领域
4,NaN,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,773570,NaN
...,...,...,...,...,...,...,...,...,...,...
32129,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,773640,"Nikita ""Ghost_RUS"""
32130,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,733530,Sacada
32131,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,610660,Laush Dmitriy Sergeevich
32132,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,658870,"xropi,stev3ns"


In [21]:
df_games.isnull().sum()

# como se repiten los valores eliminaremos la columna con mas elementos faltantes, que es genres

publisher       8051
genres          3282
app_name           0
url                0
release_date    2066
tags             162
specs            669
price           1377
id                 0
developer       3298
dtype: int64

In [22]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32134 entries, 0 to 32133
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32134 non-null  object
 3   url           32134 non-null  object
 4   release_date  30068 non-null  object
 5   tags          31972 non-null  object
 6   specs         31465 non-null  object
 7   price         30757 non-null  object
 8   id            32134 non-null  object
 9   developer     28836 non-null  object
dtypes: object(10)
memory usage: 2.5+ MB


### Observamos previamente que genres y tags tienen semejanza, ademas de mucha data faltante, entonces imputaremos la data faltante en tags, la cual sera nuestra nueva columna de genres

In [23]:
df_genero = df_games[(df_games["genres"].isna()) & (df_games['tags'].isna())]
df_genero.shape

(138, 10)

### Concluimos para esta caso entonces, que será más útil utilizar la columna tags, pues contiene menor cantidad de valores faltantes

In [24]:
df_games.drop(["tags"],inplace=True,axis=1)  # Eliminamos la columna tags
df_games


,publisher,genres,app_name,url,release_date,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,[Single-player],0.99,767400,彼岸领域
4,NaN,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN
...,...,...,...,...,...,...,...,...,...
32129,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Single-player, Steam Achievements]",1.99,773640,"Nikita ""Ghost_RUS"""
32130,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Single-player, Steam Achievements, Steam Clou...",4.99,733530,Sacada
32131,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Single-player, Steam Achievements, Steam Trad...",1.99,610660,Laush Dmitriy Sergeevich
32132,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Single-player, Steam Achievements, Steam Cloud]",4.99,658870,"xropi,stev3ns"


In [25]:
df_games.isnull().sum()

publisher       8051
genres          3282
app_name           0
url                0
release_date    2066
specs            669
price           1377
id                 0
developer       3298
dtype: int64

### Observaremos la columna release date, pues es importante obtener en la fecha en años

In [26]:
tipo_release = df_games["release_date"].apply(type)
contador_tipo_release = Counter(tipo_release)

for tipo,cantidad in contador_tipo_release.items():
    print(tipo,cantidad)

<class 'str'> 30068
<class 'float'> 2066


In [27]:
filtro = df_games["release_date"].apply(lambda x: isinstance(x,float))
filtro2 = df_games[filtro]
filtro2

# Observamos que para requerimientos de las consignas, no es necesario cambiar el tipo de dato

,publisher,genres,app_name,url,release_date,specs,price,id,developer
4,NaN,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN
11,NaN,NaN,Icarus Six Sixty Six,http://store.steampowered.com/app/724910/Icaru...,NaN,"[Single-player, HTC Vive, Tracked Motion Contr...",Free,724910,NaN
19,NaN,NaN,After Life VR,http://store.steampowered.com/app/772590/After...,NaN,"[Single-player, HTC Vive, Tracked Motion Contr...",4.99,772590,NaN
20,NaN,NaN,Kitty Hawk,http://store.steampowered.com/app/640250/Kitty...,NaN,"[Single-player, Steam Leaderboards, HTC Vive, ...",2.99,640250,NaN
22,NaN,NaN,Mortars VR,http://store.steampowered.com/app/711440/Morta...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",0.99,711440,NaN
...,...,...,...,...,...,...,...,...,...
32076,NaN,NaN,SpaceWalker,http://store.steampowered.com/app/705860/Space...,NaN,"[Single-player, HTC Vive, Oculus Rift, Tracked...",Free,705860,NaN
32083,NaN,"[Action, Casual, Indie]",BOMBFEST,http://store.steampowered.com/app/729640/BOMBF...,NaN,"[Single-player, Multi-player, Local Multi-Play...",NaN,729640,The BOMBFEST Team
32084,NaN,NaN,LIV Client,http://store.steampowered.com/app/755540/LIV_C...,NaN,"[Steam Workshop, Steam Cloud, HTC Vive, Oculus...",NaN,755540,NaN
32087,rechargecomplete,"[Action, Adventure, Indie, Early Access]",RECHARGE COMPLETE,http://store.steampowered.com/app/708070/RECHA...,NaN,"[Single-player, Steam Achievements]",NaN,708070,rechargecomplete


In [28]:
filtro2["release_date"].isna().sum()

# Observamos que los 2066 valores obtenidos como flotante en realidad son valores NaN

2066

In [29]:
df_games[(df_games["price"]=="Free to Play") |(df_games["price"]=="Free")]

,publisher,genres,app_name,url,release_date,specs,price,id,developer
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290,Poolians.com
11,NaN,NaN,Icarus Six Sixty Six,http://store.steampowered.com/app/724910/Icaru...,NaN,"[Single-player, HTC Vive, Tracked Motion Contr...",Free,724910,NaN
60,Unknown Worlds Entertainment,"[Casual, Indie]",Zen of Sudoku,http://store.steampowered.com/app/4900/Zen_of_...,2006-12-14,[Single-player],Free,4900,Unknown Worlds Entertainment
190,Tripwire Interactive,[Action],Mare Nostrum,http://store.steampowered.com/app/1230/Mare_No...,2008-10-17,"[Multi-player, Mods, Steam Achievements]",Free,1230,Sandstorm Productions
247,Darklight Games,"[Action, Simulation]",Darkest Hour: Europe '44-'45,http://store.steampowered.com/app/1280/Darkest...,2009-06-09,"[Multi-player, Mods, Partial Controller Suppor...",Free,1280,Darklight Games
...,...,...,...,...,...,...,...,...,...
31992,Sekai Project,"[Free to Play, Simulation]",Six Days of Snow,http://store.steampowered.com/app/754810/Six_D...,2017-12-29,"[Single-player, Steam Achievements]",Free,754810,ebi-hime
32006,Engin Mercan,"[Action, Casual, Free to Play, Indie]",Bloodworks,http://store.steampowered.com/app/717990/Blood...,2018-01-01,"[Single-player, Steam Achievements, Steam Work...",Free,717990,Engin Mercan
32076,NaN,NaN,SpaceWalker,http://store.steampowered.com/app/705860/Space...,NaN,"[Single-player, HTC Vive, Oculus Rift, Tracked...",Free,705860,NaN
32100,KUMA GAMES,[Simulation],High School Simulator,http://store.steampowered.com/app/754620/High_...,2018-01-03,"[Single-player, Partial Controller Support]",Free,754620,KUMA GAMES


In [30]:
def procesa_string(elemento):
    if isinstance(elemento,str):
        # Convertimos a minusculas
        elemento = elemento.lower().strip()
        # Ahora reemplazamos los valores
        if elemento=="free to play" or elemento=="free" or elemento =="free to use" or elemento == "play for free!" or elemento=="install now" or elemento=="free demo" or elemento=="free mod":
            elemento = 0
    return elemento

In [31]:
df_games["price"] = df_games["price"].apply(lambda x: procesa_string(x))
df_games

,publisher,genres,app_name,url,release_date,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",0,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",0,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,[Single-player],0.99,767400,彼岸领域
4,NaN,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN
...,...,...,...,...,...,...,...,...,...
32129,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Single-player, Steam Achievements]",1.99,773640,"Nikita ""Ghost_RUS"""
32130,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Single-player, Steam Achievements, Steam Clou...",4.99,733530,Sacada
32131,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Single-player, Steam Achievements, Steam Trad...",1.99,610660,Laush Dmitriy Sergeevich
32132,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Single-player, Steam Achievements, Steam Cloud]",4.99,658870,"xropi,stev3ns"


In [32]:
def cambiar_tipo(cadena):
    """
    Esta funcion ayuda a transformar los datos de un string para los diferentes casos, mostrando una data mas limpia
    """
    if (cadena=="play warmachine: tactics demo") or (cadena=='free mod'):
        cadena=0
    elif isinstance(cadena,str):
        try:
            cadena = float(cadena)
        except ValueError:
            cadena=0
    elif isinstance(cadena,int) or isinstance(cadena,float):
        pass
    elif cadena==None:
        cadena=0
    return cadena


# ETL DF_GAMES

In [33]:
df_games["price"] = df_games["price"].apply(lambda x: cambiar_tipo(x))
df_games["id"] = df_games["id"].astype(str)
df_games["id"] = df_games["id"].apply(lambda x: x.rstrip("0").rstrip('.') if '.' in x else x)

In [35]:
df_games

,publisher,genres,app_name,url,release_date,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",0.00,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",0.00,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,[Single-player],0.99,767400,彼岸领域
4,NaN,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN
...,...,...,...,...,...,...,...,...,...
32129,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Single-player, Steam Achievements]",1.99,773640,"Nikita ""Ghost_RUS"""
32130,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Single-player, Steam Achievements, Steam Clou...",4.99,733530,Sacada
32131,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Single-player, Steam Achievements, Steam Trad...",1.99,610660,Laush Dmitriy Sergeevich
32132,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Single-player, Steam Achievements, Steam Cloud]",4.99,658870,"xropi,stev3ns"


### Finalmente procedemos a la transformacion de este Dataframe a parquet para el uso del mismo en la FastAPI

## Hasta aquí concluye la limpieza de los datos en el df de games

In [37]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32134 entries, 0 to 32133
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32134 non-null  object 
 3   url           32134 non-null  object 
 4   release_date  30068 non-null  object 
 5   specs         31465 non-null  object 
 6   price         30757 non-null  float64
 7   id            32134 non-null  object 
 8   developer     28836 non-null  object 
dtypes: float64(1), object(8)
memory usage: 2.2+ MB


In [38]:
# Hacemos One-Hot Encoding para cada categoria

df_expandido = df_games["genres"].str.join("|").str.get_dummies().add_prefix("genre_")
df_genres = pd.concat([df_games,df_expandido],axis=1)
df_genres.drop("genres",axis=1,inplace=True) # ESTE ES EL DATAFRAME DE GENRES

In [41]:
df_genres

,publisher,app_name,url,release_date,specs,price,id,developer,genre_Accounting,genre_Action,...,genre_Photo Editing,genre_RPG,genre_Racing,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Web Publishing
0,Kotoshiro,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,[Single-player],4.99,761140,Kotoshiro,0,1,...,0,0,0,1,0,0,1,0,0,0
1,"Making Fun, Inc.",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",0.00,643980,Secret Level SRL,0,0,...,0,1,0,0,0,0,1,0,0,0
2,Poolians.com,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",0.00,670290,Poolians.com,0,0,...,0,0,0,1,0,1,0,0,0,0
3,彼岸领域,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,[Single-player],0.99,767400,彼岸领域,0,1,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32129,Ghost_RUS Games,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Single-player, Steam Achievements]",1.99,773640,"Nikita ""Ghost_RUS""",0,0,...,0,0,0,1,0,0,1,0,0,0
32130,Sacada,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Single-player, Steam Achievements, Steam Clou...",4.99,733530,Sacada,0,0,...,0,0,0,0,0,0,1,0,0,0
32131,Laush Studio,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Single-player, Steam Achievements, Steam Trad...",1.99,610660,Laush Dmitriy Sergeevich,0,0,...,0,0,1,1,0,0,0,0,0,0
32132,SIXNAILS,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Single-player, Steam Achievements, Steam Cloud]",4.99,658870,"xropi,stev3ns",0,0,...,0,0,0,0,0,0,0,0,0,0


### CREAR ARCHIVO PARA DF_GENRES

## Ahora procedemos con la limpieza de los datos del segundo archivo de reviews

In [43]:
rows = []

with open ('australian_user_reviews.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_user_reviews = pd.DataFrame(rows)

In [44]:
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


### Primero verificamos los valores nulos y faltantes en user_id que es la columna principal la cual se relacionara con el df de items

In [45]:
df_user_reviews.isnull().sum()

# Observamos que no hay valores nulos

user_id     0
user_url    0
reviews     0
dtype: int64

In [46]:
# Usar la función explode para convertir la columna "reviews" en un DataFrame independiente
expanded_df_reviews = df_user_reviews.explode('reviews', ignore_index=True)

# Expandir las claves de los diccionarios en columnas
expanded_df_reviews = pd.concat([expanded_df_reviews, expanded_df_reviews['reviews'].apply(pd.Series)], axis=1)

# Eliminar la columna 'reviews' original
expanded_df_reviews.drop(columns='reviews', inplace=True)

expanded_df_reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D,NaN


In [47]:
expanded_df_reviews.drop(columns=[0,'last_edited','helpful','funny'],inplace = True)
expanded_df_reviews

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730,True,:D


In [48]:
expanded_df_reviews.isnull().sum()

user_id       0
user_url      0
posted       28
item_id      28
recommend    28
review       28
dtype: int64

In [49]:
expanded_df_reviews[expanded_df_reviews.isnull().sum(axis=1)>=4]

# Observamos que hay algunas filas nulas, las cuales eliminaremos del df

,user_id,user_url,posted,item_id,recommend,review
137,gdxsd,http://steamcommunity.com/id/gdxsd,NaN,NaN,NaN,NaN
177,76561198094224872,http://steamcommunity.com/profiles/76561198094...,NaN,NaN,NaN,NaN
2559,76561198021575394,http://steamcommunity.com/profiles/76561198021...,NaN,NaN,NaN,NaN
10080,cmuir37,http://steamcommunity.com/id/cmuir37,NaN,NaN,NaN,NaN
13767,Jaysteeny,http://steamcommunity.com/id/Jaysteeny,NaN,NaN,NaN,NaN
15493,ML8989,http://steamcommunity.com/id/ML8989,NaN,NaN,NaN,NaN
19184,76561198079215291,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN
20223,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN
25056,76561198061996985,http://steamcommunity.com/profiles/76561198061...,NaN,NaN,NaN,NaN
26257,76561198108286351,http://steamcommunity.com/profiles/76561198108...,NaN,NaN,NaN,NaN


In [50]:
expanded_df_reviews = expanded_df_reviews[expanded_df_reviews.isnull().sum(axis=1)<2]
expanded_df_reviews.reset_index(drop = True, inplace=True)

expanded_df_reviews.head()

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [51]:
expanded_df_reviews['review'].value_counts()

review
good game                                                                                                                           100
good                                                                                                                                 98
great game                                                                                                                           83
10/10                                                                                                                                76
Great game                                                                                                                           50
                                                                                                                                   ... 
Idk how to close this game please help                                                                                                1
Great game! Best Zombie shooter ever made

### Aqui procedemos a separar la data para poder trabajarla en base al formato requerido para reviews

In [52]:
def separar_elementos(cadena):
    cadena = cadena.strip()
    cadena = cadena.replace(",","").replace(".","")
    lista_espacios = cadena.split(" ")
    lista = lista_espacios[1:]
    return lista

In [53]:
expanded_df_reviews["posted"] = expanded_df_reviews["posted"].apply(lambda x: separar_elementos(x))
expanded_df_reviews

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[November, 5, 2011]",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[July, 15, 2011]",22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[April, 21, 2011]",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"[June, 24, 2014]",251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"[September, 8, 2013]",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[July, 10]",70,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[July, 8]",362890,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"[July, 3]",273110,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"[July, 20]",730,True,:D


In [54]:
def saca_mes(mes):
    # Devuelve el mes en formato numero para consideraciones de la consigna
    lista_meses=["January","February","March","April","May","June","July","August","September","October","November","December"]
    for pos,i in enumerate(lista_meses):
        if i==mes:
            return str(pos+1)

In [55]:
# Empezaremos tratando la data

def transforma_formato(lista):
    if len(lista)>2:
        mes_numero = saca_mes(lista[0]).zfill(2)
        dia = lista[1].zfill(2)
        anio = lista[2]
        return f"{anio}-{mes_numero}-{dia}"
    else:
        return None


In [56]:
expanded_df_reviews["posted"] = expanded_df_reviews["posted"].apply(lambda x: transforma_formato(x))
expanded_df_reviews

# Se procede a poner en formato date la fila posted, pero solo para aquellos que contengan el año, pues para aquellos que no se los deja en fomato original

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,70,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,362890,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,273110,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,730,True,:D


### Aca aplicamos NLTK y hacemos tratamiento de los datos

In [57]:
expanded_df_reviews['review'].dropna(inplace=True)
expanded_df_reviews

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,70,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,362890,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,273110,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,730,True,:D


In [58]:
expanded_df_reviews['review'] = expanded_df_reviews['review'].apply(lambda x: x.lower())

In [59]:
def cambio_palabra(string_separado):
    nuevo_string = string_separado.replace(". ",",")
    return nuevo_string

In [60]:
expanded_df_reviews['review'] = expanded_df_reviews['review'].apply(cambio_palabra)
expanded_df_reviews['review']

0        simple yet with great replayability,in my opin...
1                     it's unique and worth a playthrough.
2        great atmosphere,the gunplay can be a bit chun...
3        i know what you think when you see this title ...
4        for a simple (it's actually not all that simpl...
                               ...                        
59300    a must have classic from steam definitely wort...
59301    this game is a perfect remake of the original ...
59302    had so much fun plaing this and collecting res...
59303                                                   :d
59304                                       so much fun :d
Name: review, Length: 59305, dtype: object

In [61]:
import nltk
nltk.data.path.append("/path/to/nltk_data")
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dennys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dennys\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dennys\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [62]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
analizador = SentimentIntensityAnalyzer()

In [63]:
def analysis_sentiments(palabra_review):
    if palabra_review !="" or palabra_review==np.NAN:
        sentences = tokenizer.tokenize(palabra_review)
        resultante = 0
        sentimiento = 0
        for sentence in sentences:
            scores = analizador.polarity_scores(sentence)
            for key in scores:
                if scores[key]>resultante and key!="compound":
                    resultante = scores[key]
                    sentimiento = key
        if sentimiento=="neu":
            return 1
        elif sentimiento=="pos":
            return 2
        else:
            return 0
    else:
        return 1

In [64]:
expanded_df_reviews["review"]=expanded_df_reviews["review"].apply(lambda x: analysis_sentiments(x))

In [65]:
expanded_df_reviews

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,True,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,True,1
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,True,1
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,True,1
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,True,1
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,70,True,1
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,None,362890,True,1
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,273110,True,1
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,None,730,True,2


## Ahora que tenemos la data limpia, procedemos a pasarla a formato parquet para su lectura

In [68]:
expanded_df_reviews.head()

,user_id,user_url,posted,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,True,1
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,True,1
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,True,1
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,True,1
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,True,1


In [70]:
df_genres.head()

,publisher,app_name,url,release_date,specs,price,id,developer,genre_Accounting,genre_Action,...,genre_Photo Editing,genre_RPG,genre_Racing,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Web Publishing
0,Kotoshiro,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,[Single-player],4.99,761140,Kotoshiro,0,1,...,0,0,0,1,0,0,1,0,0,0
1,"Making Fun, Inc.",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",0.00,643980,Secret Level SRL,0,0,...,0,1,0,0,0,0,1,0,0,0
2,Poolians.com,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",0.00,670290,Poolians.com,0,0,...,0,0,0,1,0,1,0,0,0,0
3,彼岸领域,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,[Single-player],0.99,767400,彼岸领域,0,1,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


## Ahora procedemos a la lectura del ultimo archivo, relacionado a los items

In [75]:
df_users_items = pd.read_parquet("australian_users_items.parquet")
df_users_items

,user_id,steam_id,user_url,item_id,item_name,playtime_forever
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6
1,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0
2,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7
3,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0
4,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0
...,...,...,...,...,...,...
5153204,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,346330,BrainBread 2,0
5153205,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0
5153206,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3
5153207,76561198329548331,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4


In [3]:
df_users_items.isnull().sum()

user_id             0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
dtype: int64

In [4]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   steam_id          object
 2   user_url          object
 3   item_id           object
 4   item_name         object
 5   playtime_forever  int64 
dtypes: int64(1), object(5)
memory usage: 235.9+ MB


### Pasamos la data a su respectivo tipo de dato

In [5]:
df_users_items["user_url"] = df_users_items["user_url"].astype(str)
df_users_items["item_id"] = df_users_items["item_id"].astype(int)
df_users_items["item_name"] = df_users_items["item_name"].astype(str)
df_users_items["playtime_forever"] = df_users_items["playtime_forever"].astype(int)

In [6]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   steam_id          object
 2   user_url          object
 3   item_id           int32 
 4   item_name         object
 5   playtime_forever  int32 
dtypes: int32(2), object(4)
memory usage: 196.6+ MB
